In [ ]:
%load_ext autoreload
%autoreload 2

import platformdirs
import pathlib


from katago.game.board import Board

from katago_onnx.utils import load_model
from katago_onnx.utils import load_sgf
from katago_onnx.utils import featurize
from katago_onnx.utils import run_inference
from katago_onnx.utils import get_top_moves


cache_dir = pathlib.Path(platformdirs.user_cache_dir("katago-onnx"))

# Load model
network_name = "kata1-b28c512nbt-adam-s11165M-d5387M"
model_path = cache_dir / network_name / "model.ckpt"
model = load_model(model_path, device="cpu")

In [9]:
target_move = 11

# Path to SGF
sgf_path = "../sgf/game-19-white-38.sgf"
# sgf_path = "../sgf/game-13-white-12.sgf"
# sgf_path = "../sgf/game-9-white-19.sgf"

# Load game state from SGF
gamestate = load_sgf(sgf_path, target_move=target_move)

# Featurize the game state
features, bin_input, global_input = featurize(gamestate, model)

current_player_name = "Black" if gamestate.board.pla == Board.BLACK else "White"
print(f"Current player to play: {current_player_name}")

bin_input.shape, global_input.shape

Loaded SGF with 282 moves.
Game size: 19
Replayed to move 11
Current player to play: White


(torch.Size([1, 22, 19, 19]), torch.Size([1, 19]))

In [10]:
policy_probs, value_probs, winrate, score_lead = run_inference(model, bin_input, global_input)

winrate, score_lead

(np.float32(0.5373279), 0.5365096032619476)

In [11]:
top_moves = get_top_moves(policy_probs, gamestate, features, top_k=10)

top_moves


,loc,x,y,prob,move_str
0,98,17,3,0.997650,S16
1,336,15,15,0.000053,Q4
2,357,16,16,0.000048,R3
3,356,15,16,0.000043,Q3
4,123,2,5,0.000037,C14
5,283,2,13,0.000035,C6
6,317,16,14,0.000034,R5
7,355,14,16,0.000034,P3
8,104,3,4,0.000034,D15
9,346,5,16,0.000033,F3
